In [79]:
import pandas as pd
import numpy as np
import glob
from datetime import datetime

In [95]:
path = '../data/weatherdata'
all_files = glob.glob(path + "/*.csv")

li = []

for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)

weather_df = pd.concat(li, axis=0, ignore_index=True)

/Users/danielchoy/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (43,44,48,53,57,58) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/danielchoy/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (45,46,50,55,59,88,90) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/danielchoy/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (42,43,44,45,56,57,64,90) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/danielchoy/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (42,43,44,45,49,54,58,59) have mi

In [46]:
pd.reset_option('display.max_columns', None)

Convert wind speed from meters/second to miles/hour. The conversion can be done using the following formula:

$$s(mph) = \frac{s(m/s) \times 3600}{1000 \times 1.6}$$

https://www.weather.gov/pqr/wind

In [100]:
def to_milesperhour(num):
    """A function to convert wind speed from meters/second to miles/hour. The conversion is done by using the 
    above formula
    """
    convert = (num * 3600)/(1000 * 1.6)
    return convert

In [99]:
def clean_weatherdata(df):
    """This function will clean the weather data from any given year or years (the merged weather data)
    Weather data was obtained from NOAA( National Cceanic and Atmospheric Administration ) 
    https://www.ncei.noaa.gov/data/global-hourly/archive/csv/.
    Additional websites were also used as references in order to interpret the numbers from the data and to
    engineer new features such as windy and rainy. The column windy and its values (breeze, gale, force...)
    were all based on information from NOAA https://www.weather.gov/pqr/wind. 
    https://www.visualcrossing.com/resources/documentation/weather-data/how-we-process-integrated-surface-database-historical-weather-data/
    was used to interpret the numbers of the data.
    """
    # Make a copy
    nyweather= df.copy()
    
    # Select the columns that will be used
    nyweather = nyweather[['DATE', 'SOURCE', 'LATITUDE', 'LONGITUDE', 'ELEVATION', 'NAME', 'WND', 'TMP',
             'DEW', 'SLP', 'AA1','AA2']]
    
    # Clean DATE column
    nyweather['DATE'] = nyweather['DATE'].str.replace('T', ' ')
    
    # Clean TMP (temperature) column. Please refer to 
    # Interpreting the numbers are based on 
    #https://www.visualcrossing.com/resources/documentation/weather-data/how-we-process-integrated-surface-database-historical-weather-data/

    nyweather['TMP'] = nyweather['TMP'].astype(str)
    nyweather['TMP'] = nyweather['TMP'].str.slice(0, -2)
    nyweather['TMP'] = nyweather['TMP'].str.replace('+','')
    nyweather['TMP'] = nyweather['TMP'].apply(lambda x: '0' if x == '9999')
    nyweather['TMP'] = nyweather['TMP'].astype(int)
    nyweather['TMP'] = nyweather['TMP']/10
    
    print('complete part 1')
    
    # Clean WND column
    # Please refer to https://www.weather.gov/pqr/wind
    nyweather['WND'] = nyweather['WND'].astype(str)
    nyweather = pd.concat([nyweather, nyweather.WND.str.split(',', expand = True)],1)
    nyweather = nyweather.rename(columns={3:'wind_speed'})
    nyweather['wind_speed'] = nyweather['wind_speed'].astype(int)
    nyweather['wind_speed'] = nyweather['wind_speed']/10
    nyweather['wind_speed'] = nyweather['wind_speed'].map(to_milesperhour)
    
    # Engineered a new feature called windy
    nyweather['windy'] = nyweather['wind_speed'].apply(lambda x: 'calm' if x < 4 else 'breeze' if x < 12 else \
                                        'moderate breeze' if x < 24 else 'strong breeze' if x < 31 else \
                                        'gale' if x < 63 else 'storm force')
    nyweather = nyweather[['DATE', 'SOURCE', 'LATITUDE', 'LONGITUDE', 'ELEVATION', 'NAME', 'WND', 'TMP',\
                           'DEW', 'SLP', 'AA1','AA2', 'wind_speed', 'windy']]
    
    print('complete part 2')
    
    # Clean the AA1 column that has precipitation information
    nyweather['AA1'] = nyweather['AA1'].fillna('0,0,0,0')
    nyweather['AA1'] = nyweather['AA1'].astype(str)
    nyweather = pd.concat([nyweather, nyweather.AA1.str.split(',', expand = True)],1)
    nyweather = nyweather.rename(columns={1:'precipitation'})
    nyweather['precipitation'] = nyweather['precipitation'].astype(int)
    nyweather['precipitation'] = nyweather['precipitation']/10
    nyweather = nyweather[['DATE', 'SOURCE', 'LATITUDE', 'LONGITUDE', 'ELEVATION', 'NAME', 'WND', 'TMP',\
                           'DEW', 'SLP', 'wind_speed', 'windy', 'precipitation']]
    
    # Engineered a new feature called rainy
    nyweather['rainy'] = nyweather['precipitation'].apply(lambda x: 'rainy' if x > 0 else 'not rainy')
    
    
    # Clean DATE column
    # Generate new features - month, date, hour and month2
    nyweather['DATE'] = nyweather['DATE'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))
    nyweather['month'] = nyweather.DATE.dt.month
    nyweather['date'] = nyweather.DATE.dt.date
    nyweather['hour'] = nyweather.DATE.dt.hour
    nyweather['month2'] = nyweather.DATE.dt.month.apply(lambda x: 'Jan' if x == 1 else 'Feb' if x == 2 else \
    'Mar' if x == 3 else 'Apr' if x == 4 else 'May' if x == 5 else 'Jun' if x == 6 else 'Jul' if x == 7 else \
    'Aug' if x == 8 else 'Sep' if x == 9 else 'Oct' if x == 10 else 'Nov' if x == 11 else 'Dec' )
    
    # Generate cleaned dataframe
    return nyweather

In [97]:
weather_df = clean_weatherdata(weather_df)

complete part 1
complete part 2


In [98]:
weather_df

,DATE,SOURCE,LATITUDE,LONGITUDE,ELEVATION,NAME,WND,TMP,DEW,SLP,wind_speed,windy,precipitation,rainy,month,date,hour,month2
0,2017-01-01 00:00:00,4,40.77944,-73.88035,3.4,"LAGUARDIA AIRPORT, NY US","200,1,N,0077,1",6.7,"-0028,1","10143,1",17.325,moderate breeze,0.0,not rainy,1,2017-01-01,0,Jan
1,2017-01-01 00:51:00,7,40.77944,-73.88035,3.4,"LAGUARDIA AIRPORT, NY US","220,5,N,0051,5",6.7,"-0039,5","10139,5",11.475,breeze,0.0,not rainy,1,2017-01-01,0,Jan
2,2017-01-01 01:51:00,7,40.77944,-73.88035,3.4,"LAGUARDIA AIRPORT, NY US","210,5,N,0051,5",6.7,"-0044,5","10131,5",11.475,breeze,0.0,not rainy,1,2017-01-01,1,Jan
3,2017-01-01 02:51:00,7,40.77944,-73.88035,3.4,"LAGUARDIA AIRPORT, NY US","200,5,N,0051,5",6.7,"-0044,5","10128,5",11.475,breeze,0.0,not rainy,1,2017-01-01,2,Jan
4,2017-01-01 03:00:00,4,40.77944,-73.88035,3.4,"LAGUARDIA AIRPORT, NY US","200,1,N,0051,1",6.7,"-0044,1","10128,1",11.475,breeze,0.0,not rainy,1,2017-01-01,3,Jan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109768,2019-12-31 20:51:00,7,40.77944,-73.88035,3.4,"LAGUARDIA AIRPORT, NY US","240,5,N,0046,5",7.2,"+0011,5","10028,5",10.350,breeze,0.0,not rainy,12,2019-12-31,20,Dec
109769,2019-12-31 21:00:00,4,40.77944,-73.88035,3.4,"LAGUARDIA AIRPORT, NY US","240,1,N,0046,1",7.2,"+0011,1","10028,1",10.350,breeze,0.0,not rainy,12,2019-12-31,21,Dec
109770,2019-12-31 21:51:00,7,40.77944,-73.88035,3.4,"LAGUARDIA AIRPORT, NY US","250,5,N,0062,5",7.2,"+0000,5","10033,5",13.950,moderate breeze,0.0,not rainy,12,2019-12-31,21,Dec
109771,2019-12-31 22:51:00,7,40.77944,-73.88035,3.4,"LAGUARDIA AIRPORT, NY US","240,5,N,0046,5",7.2,"+0006,5","10036,5",10.350,breeze,0.0,not rainy,12,2019-12-31,22,Dec


In [1]:
daily = weather_df.groupby(['date'])[['precipitation','wind_speed']].\
mean().rename(columns={'precipitation':'avg_precip', 'wind_speed': 'avg_wind_speed'}).reset_index()

NameError: name 'weather_df' is not defined